In [1]:
from config import *
from logger import getLogger  

import pandas as pd
import numpy as np
import os
import datetime
import warnings
warnings.filterwarnings('ignore')
import pickle
from statistics import mode
import statsmodels
import statsmodels.api as sm
from pmdarima.arima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.tsa.holtwinters as ets
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from tbats import TBATS, BATS
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

from sklearn import metrics
from joblib import Parallel, delayed

from ta import add_all_ta_features

#import boto3,botocore
#import pytz
from datetime import timedelta, datetime
import time
import xlsxwriter

from utility import modelling_utility_func
from utility import feature_selection_func
from utility import feature_extraction_func
from utility import preprocessing_func
from utility import ts_multivariate_func
from utility import ts_univariate_func

from sklearn.feature_selection import RFE, RFECV
from sklearn.preprocessing import StandardScaler,RobustScaler, MinMaxScaler

from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso ,BayesianRidge ,SGDRegressor ,Lars , LassoLars, LassoLarsIC
from sklearn.linear_model import OrthogonalMatchingPursuit,ARDRegression ,MultiTaskElasticNet , MultiTaskLasso, HuberRegressor,RANSACRegressor ,TheilSenRegressor , PoissonRegressor 
from sklearn.linear_model import TweedieRegressor,GammaRegressor,PassiveAggressiveRegressor,enet_path,lars_path,enet_path

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor

2022-09-06 09:19:07.609361: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-06 09:19:07.609424: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from utility import Data_Prep_func


all_feature_df_full = Data_Prep_func.training_data(INPUT_FILE_X,NO_OF_SHEETS,INPUT_FILE_Y)
all_feature_df = all_feature_df_full[all_feature_df_full.index<=TAKE_ACTUAL_TILL]

In [ ]:
OPTUNA_EARLY_STOPING = 15

class EarlyStoppingExceeded(optuna.exceptions.OptunaError):
    early_stop = OPTUNA_EARLY_STOPING
    early_stop_count = 0
    best_score = None

def early_stopping_opt(study, trial):
    if EarlyStoppingExceeded.best_score == None:
        EarlyStoppingExceeded.best_score = study.best_value

    if study.best_value < EarlyStoppingExceeded.best_score:
        EarlyStoppingExceeded.best_score = study.best_value
        EarlyStoppingExceeded.early_stop_count = 0
    else:
        if EarlyStoppingExceeded.early_stop_count > EarlyStoppingExceeded.early_stop:
            EarlyStoppingExceeded.early_stop_count = 0
            best_score = None
            raise EarlyStoppingExceeded()
        else:
            EarlyStoppingExceeded.early_stop_count=EarlyStoppingExceeded.early_stop_count+1
    #print(f'EarlyStop counter: {EarlyStoppingExceeded.early_stop_count}, Best score: {study.best_value} and {EarlyStoppingExceeded.best_score}')
    return

In [3]:
from optuna.pruners import BasePruner
from optuna.structs import TrialState
class RepeatPruner(BasePruner):
    def prune(self, study, trial):
        # type: (Study, FrozenTrial) -> bool

        trials = study.get_trials(deepcopy=False)
        
        numbers=np.array([t.number for t in trials])
        bool_params= np.array([trial.params==t.params for t in trials]).astype(bool)
        
        #Don´t evaluate function if another with same params has been/is being evaluated before this one
        if np.sum(bool_params)>1:
            if trial.number>np.min(numbers[bool_params]):
                return True

        return False

In [4]:
# '''
# This .py file contains functions related to hyperparameter tuning
# '''
import pandas as pd
import numpy as np
from ta import add_all_ta_features
import optuna
from scipy.stats import loguniform
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

class Hyperparameter:
    """
    class to tune the hyperparameters
    
    """
    def __init__(self, model_name, error_metric,trainX,trainY,futureX,
                 test_period, backtest_period,target, n_trials=OPTUNA_TRIALS, random_state=42,
                y_log_scale = True):
        self.model_name = model_name
        self.error_metric = error_metric
        self.trainX = trainX
        self.trainY = trainY
        self.target = target
        self.trainY.index.freq = self.trainY.index.inferred_freq
        self.y_log_scale = y_log_scale
        if self.y_log_scale:
            self.trainY = np.log(self.trainY) # ONLY Y LOG
        self.futureX = futureX
        self.test_period = test_period 
        self.backtest_period = backtest_period 
        
        self.n_trials = n_trials # only valid for optuna
        self.random_state = random_state
        
        self.cv_error = 10000000 #This value is set to avoid Optuna NaN error in non-converging cases
        self.params = None
        self.best_params = None
        self.search_space= None
        self.run_univariate = True
        self.run_hyperparameter = False
          
    def init_search_space(self):
        space_size=5
        if self.model_name == "Ridge": 
            search_space = {'normalize': [True, False],
                             'alpha':list(loguniform.rvs(1,50,size=space_size, random_state=self.random_state))}
            
        elif self.model_name == "XGB":  
            search_space = {'max_depth':list(range(20,50,1)),
                            'min_child_weight':list(loguniform.rvs(1,2,size=space_size,random_state=self.random_state)),
                            'learning_rate' : list(loguniform.rvs(0.7,0.8,size=space_size,random_state=self.random_state)),
                            'subsample' : list(loguniform.rvs(0.8,0.9,size=space_size,random_state=self.random_state)),
                            'colsample_bytree' : list(loguniform.rvs( 0.1, 0.2,size=space_size,random_state=self.random_state)),
                            'gamma' : list(range(35,50,1)),
                            'reg_lambda' : list(loguniform.rvs(0.1,0.5,size=space_size,random_state=self.random_state))} 
            
        elif self.model_name == 'LGBM':
            search_space = {'max_depth':list(range(2, 12)),
                            'num_leaves' : list(range(30, 100)),
                            'min_child_weight':list(loguniform.rvs(1, 200,size=space_size,random_state=self.random_state)),
                            'learning_rate' : list(loguniform.rvs(0.01,0.9,size=space_size,random_state=self.random_state)),
                            'subsample' : list(loguniform.rvs(0.7,1,size=space_size,random_state=self.random_state)),
                            'colsample_bytree' : list(loguniform.rvs(0.01,1,size=space_size,random_state=self.random_state)),
                            'reg_lambda' : list(loguniform.rvs(0.01,30,size=space_size,random_state=self.random_state))}
            
        elif self.model_name == 'SGDR':
            search_space =  {
                            'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3], # learning rate
                            'max_iter': [100,10000,1000], # number of epochs
                            'penalty': ['l2', 'l1', 'elasticnet']}
            
        elif self.model_name == 'BR':
            search_space =  {'alpha_init':[1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.9],
                  'lambda_init': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-9]}
            
        elif self.model_name == 'ElasticNet':
            search_space =  {"max_iter": [1, 5, 10],
                          "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                          "l1_ratio": np.arange(0.0, 1.0, 0.1)}
            
        elif self.model_name == 'Lasso':
            search_space = {
                          "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}
        
        elif self.model_name == 'ARDR':
            search_space =  { 'alpha_1': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3], # learning rate
                             'n_iter' : [100,20,1000,10000] }
            
        elif self.model_name == 'HR':
            search_space = {'epsilon': [1,1.5,2,2.5,3], # learning rate
                            'max_iter': [100,20,1000,10000],
                            'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]}
            
        elif self.model_name == 'RFR':
            search_space = {'n_estimators' : list( range(50, 200)),
                                'max_depth' :  list(range(5, 20)),
                                'min_samples_split' : list(range(2, 10)),
                                'min_samples_leaf' : list(range( 2, 10))}
            
        elif self.model_name == 'GBM':
            search_space = {'loss':['ls', 'lad', 'huber', 'quantile'],
                            'learning_rate' :list(loguniform.rvs(0.01,1,size=space_size,random_state=self.random_state)),
                            'criterion' : ['friedman_mse', 'mse', 'mae']}
            
        elif self.model_name == 'Adaboost':
            search_space = { 'n_estimators' : list(range(10,100)),
                        'learning_rate' : list(loguniform.rvs( 0.01, 0.9,size=space_size,random_state=self.random_state ))}
            
        elif self.model_name == 'Ext':
            search_space = {'n_estimators' : list(range(10,500)), 
                            'max_depth' : [-1,0,1] }

            
        else:
            print(f"HYPERPAMETER SPACE FOR {self.model_name} is not defined")
            raise Exception(f"HYPERPAMETER SPACE FOR {self.model_name} is not defined")
            
        return search_space
    
    def run_optuna(self):
        def objective(trial):
            size = 100
            # -------------------- Ridge Regression -------------------- #
            if self.model_name == "Ridge": 
                self.params = {'normalize': trial.suggest_categorical('normalize',[True,False]),
                              'alpha':trial.suggest_discrete_uniform('alpha', 1, 50,1)}

                reg = Ridge(alpha=self.params['alpha'],
                            normalize=self.params['normalize'],
                            random_state=self.random_state)
            # -------------------- XGBoost ------------------- #
            elif self.model_name == "XGB":  
                self.params = {'max_depth':trial.suggest_int('max_depth', 20, 50),
                              'min_child_weight':trial.suggest_loguniform('min_child_weight', 1, 10),
                              'learning_rate' : trial.suggest_loguniform('learning_rate', 0.5, 0.9),
                              'subsample' : trial.suggest_loguniform('subsample', 0.8, 1),
                              'colsample_bytree' : trial.suggest_loguniform('colsample_bytree', 0.1,1),
                              'gamma' : trial.suggest_uniform('gamma', 30, 50),
                              'reg_lambda' : trial.suggest_loguniform('reg_lambda', 0.1,1)}
                
            # -------------------- Light GBM -------------------- #
            elif self.model_name == "LGBM": 
                self.params = {'max_depth':trial.suggest_int('max_depth', 1, 20),
                               'num_leaves' : trial.suggest_int('num_leaves', 30, 100),
                              'min_child_weight':trial.suggest_loguniform('min_child_weight', 1, 200),
                              'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.9),
                              'subsample' : trial.suggest_loguniform('subsample', 0.7, 1),
                              'colsample_bytree' : trial.suggest_loguniform('colsample_bytree', 0.01, 1),
                              'reg_lambda' : trial.suggest_loguniform('reg_lambda', 0.01, 30)}
            
            elif self.model_name == 'SGDR':
                self.params =  {
                            'alpha': trial.suggest_categorical('alpha',[1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]), 
                            'max_iter': trial.suggest_categorical('max_iter',[100,10000,1000]), # number of epochs
                            'penalty': trial.suggest_categorical('penalty',['l2', 'l1', 'elasticnet'])}
                
            elif self.model_name == 'BR':
                self.params = {'alpha_init':trial.suggest_categorical('alpha_init',[1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.9]),
                  'lambda_init': trial.suggest_categorical('lambda_init',[1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-9])}
                
            elif self.model_name == 'ElasticNet':
                self.params = {"max_iter": trial.suggest_categorical('max_iter',[1, 5, 10]),
                          "alpha": trial.suggest_categorical('alpha',[0.0001, 0.001, 0.01, 0.1, 1, 10, 100]),
                          "l1_ratio": trial.suggest_categorical('l1_ratio',list(np.arange(0.0, 1.0, 0.1)))}
                
            elif self.model_name == 'Lasso':
                self.params = {"alpha": trial.suggest_categorical('alpha',[0.0001, 0.001, 0.01, 0.1, 1, 10, 100])}
                
            elif self.model_name == 'ARDR':
                self.params =  { 'alpha_1': trial.suggest_categorical('alpha_1',[1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]), 
                             'n_iter' :trial.suggest_categorical('n_iter', [100,20,1000,10000]) }
                
            elif self.model_name == 'HR':
                self.params = {'epsilon': trial.suggest_categorical('epsilon',[1,1.5,2,2.5,3]), # learning rate
#                             'max_iter': trial.suggest_categorical('max_iter',[100,20,1000,10000]),
                            'alpha': trial.suggest_categorical('alpha',[1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3])}
                
            elif self.model_name == 'RFR':
                self.params = {'n_estimators' : trial.suggest_int("n_estimators", 50, 200),
                                'max_depth' : trial.suggest_int("max_depth", 5, 20),
                                'min_samples_split' : trial.suggest_int("min_samples_split", 2, 10),
                                'min_samples_leaf' : trial.suggest_int("min_samples_leaf", 2, 10)}
                
            elif self.model_name == 'GBM':
                self.params = {'loss':trial.suggest_categorical("loss",['ls', 'lad', 'huber', 'quantile']),
                                'learning_rate' : trial.suggest_loguniform("learning_rate",0.01,1)}
                
            elif self.model_name == 'Adaboost':
                self.params = {'n_estimators' :trial.suggest_int('n_estimators', 10,100),
                                'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.9)}
                
            elif self.model_name == 'Ext':
                self.params =  {'n_estimators' : trial.suggest_int('n_estimators', 10, 100),
                                'max_depth' : trial.suggest_int('max_depth', 100,500)}
                
            elif self.model_name == 'AR':
                self.params = {'lags' : trial.suggest_int('lags',1,round(len(self.trainY[:-self.backtest_period])/2)-1)}
                
            elif self.model_name == 'AUTOARIMA':
                self.params = {'d' : trial.suggest_int('d',1,2)}
                
            elif self.model_name == 'ETS':
                self.params = {'trend' : trial.suggest_categorical('trend',['add','mul'])}
                
            elif self.model_name == 'Holt':
                self.params = {'damped':trial.suggest_categorical('damped',[True,False]),
                               'exponential':trial.suggest_categorical('exponential',[True,False])}
                
            elif self.model_name == 'TBATS':
                self.params = {'use_arma_errors':trial.suggest_categorical('use_arma_errors',[True,False]),
                              'use_box_cox':trial.suggest_categorical('use_box_cox',[True,False])}

            else:
                print(f"HYPERPAMETER SPACE FOR {self.model_name} is not defined")
                raise Exception(f"HYPERPAMETER SPACE FOR {self.model_name} is not defined")
                
                
            for t in trial.study.trials:
                if t.state != optuna.structs.TrialState.COMPLETE:
                    continue

                if t.params == trial.params:
                    return t.value  # Return the previous value without re-evaluating it.
#                     raise optuna.structs.TrialPruned('Duplicate parameter set')

            # Note that if duplicate parameter sets are suggested too frequently,
            # you can use the pruning mechanism of Optuna to mitigate the problem.
            # By raising `TrialPruned` instead of just returning the previous value,
            # the sampler is more likely to avoid sampling the parameters in the succeeding trials.            

            tscv_folds = modelling_utility_func.ts_traintest_split(self.trainX,NUM_FOLD,self.backtest_period)
                                                                   
            mape_losses = []
            mae_losses = []
           
            for step, (train_index, test_index) in enumerate(tscv_folds):
                major_x, minor_x = self.trainX.loc[train_index], self.trainX.loc[test_index]
                major_y, minor_y = self.trainY.loc[train_index], self.trainY.loc[test_index]
                
                if self.model_name == "Ridge":
                    reg = Ridge(random_state=self.random_state, **self.params)
                elif self.model_name == "LGBM":
                    reg = LGBMRegressor(random_state=self.random_state,num_parallel_tree=1,objective='regression',
                                        n_estimators=500, n_jobs=-1, verbosity=0, **self.params)
                elif self.model_name == 'XGB':
                    reg = XGBRegressor(**self.params)
                elif self.model_name == "SGDR":
                    reg = SGDRegressor(**self.params)
                elif self.model_name == 'BR':
                    reg = BayesianRidge(**self.params)
                elif self.model_name == 'ElasticNet':
                    reg = ElasticNet(**self.params)
                elif self.model_name == 'Lasso':
                    reg = Lasso(**self.params)
                elif self.model_name == 'ARDR':
                    reg = ARDRegression(**self.params)
                elif self.model_name == 'HR':
                    reg = HuberRegressor(**self.params)
                elif self.model_name == 'RFR':
                    reg = RandomForestRegressor(**self.params)
                elif self.model_name == 'GBM':
                    reg = GradientBoostingRegressor(**self.params)
                elif self.model_name == 'Adaboost':
                    reg = AdaBoostRegressor(**self.params)
                elif self.model_name == 'Ext':
                    reg = ExtraTreesRegressor(**self.params)      
                else:
                    break
                
                actual_val = minor_y
                reg.fit(major_x, major_y) #  Model Fit 
                pred_val = reg.predict(minor_x)
                error= modelling_utility_func.mean_absolute_percentage_error(actual_val,pred_val)
                mape_losses.append(error)
                mae_losses.append(metrics.mean_absolute_error(actual_val,pred_val)) 
                
                
            if RUN_UNIVARIATE:
                if self.model_name == 'AR':
                    val = ts_univariate_func.RUN_UNI(self.trainY,BACKTEST_PERIOD,NUM_FOLD).AR(lag=self.params['lags'],error_metric=self.error_metric)
                    mape_losses.append(val[1])
                    mae_losses.append(val[1])
                elif self.model_name == 'AUTOARIMA':
                    val = ts_univariate_func.RUN_UNI(self.trainY,BACKTEST_PERIOD,NUM_FOLD).AUTOARIMA(d=self.params['d'],error_metric=self.error_metric)
                    mape_losses.append(val[1])
                    mae_losses.append(val[1])
                elif self.model_name == 'ETS':
                    val = ts_univariate_func.RUN_UNI(self.trainY,BACKTEST_PERIOD,NUM_FOLD).ETS(t=self.params['trend'],error_metric=self.error_metric)
                    mape_losses.append(val[1])
                    mae_losses.append(val[1])
                elif self.model_name == 'Holt':
                    val = ts_univariate_func.RUN_UNI(self.trainY,BACKTEST_PERIOD,NUM_FOLD).Holt(damped=self.params['damped'],
                                                                                                exponential=self.params['exponential'],
                                                                                                error_metric=self.error_metric)
                    mape_losses.append(val[1])
                    mae_losses.append(val[1])    
                elif self.model_name == 'TBATS':
                    val = ts_univariate_func.RUN_UNI(self.trainY,BACKTEST_PERIOD,NUM_FOLD).TBATS(ar=self.params['use_arma_errors'],
                                                                                                 bc=self.params['use_box_cox'],
                                                                                                 error_metric=self.error_metric)
                    mape_losses.append(val[1])
                    mae_losses.append(val[1])
                       
            if self.error_metric == 'mape':
                return np.mean(mape_losses)
            elif self.error_metric == 'mae':
                return np.mean(mae_losses)
        
#         sampler = optuna.samplers.GridSampler(self.init_search_space())
        sampler = optuna.samplers.TPESampler(seed=99)   #Bayesian
#         sampler = optuna.samplers.RandomSampler(seed=99)
        prune=RepeatPruner()
#         prune=optuna.pruners.MedianPruner()
        study = optuna.create_study(direction='minimize',study_name="starter-experiment",sampler=sampler,pruner=prune)
    
        study.optimize(objective, n_jobs=-1, n_trials = self.n_trials,catch = (AttributeError,ValueError,ZeroDivisionError))
        
        try:
            study.optimize(objective, n_jobs=-1, n_trials = self.n_trials,catch = (AttributeError,ValueError,ZeroDivisionError),
                          callbacks=[early_stopping_opt])
            
        except EarlyStoppingExceeded:
            print(f'EarlyStopping Exceeded: No new best scores on iters {OPTUNA_EARLY_STOPING}')
        
#         plot_optimization_history(study)
        self.cv_error = study.best_value
        self.best_params = study.best_params        
        
        # BACK-TESTING ACCURACY
        
        tscv_folds = modelling_utility_func.ts_traintest_split(self.trainX,NUM_FOLD,self.backtest_period)
        model_accuracies = []
        test=[]
        for step, (train_index, test_index) in enumerate(tscv_folds):
            major_x, minor_x = self.trainX.loc[train_index], self.trainX.loc[test_index]
            major_y, minor_y = self.trainY.loc[train_index], self.trainY.loc[test_index]
                 
            actual_val = minor_y[0]
            
            if self.model_name == "Ridge":
                reg = Ridge(random_state=self.random_state, **self.best_params)
                reg.fit(major_x, major_y) 
                pred_val = reg.predict(minor_x)
                
            elif self.model_name == "LGBM":
                reg = LGBMRegressor(random_state=self.random_state,num_parallel_tree=1,objective='regression',
                                        n_estimators=500, n_jobs=-1, verbosity=0, **self.best_params)
                reg.fit(major_x, major_y) 
                pred_val = reg.predict(minor_x)
                
            elif self.model_name == 'XGB':
                reg = XGBRegressor(**self.best_params)
                reg.fit(major_x, major_y) 
                pred_val = reg.predict(minor_x)
                
            elif self.model_name == "SGDR":
                reg = SGDRegressor(**self.best_params)
                reg.fit(major_x, major_y) 
                pred_val = reg.predict(minor_x)
                
            elif self.model_name == 'BR':
                reg = BayesianRidge(**self.best_params)
                reg.fit(major_x, major_y) 
                pred_val = reg.predict(minor_x)
                
            elif self.model_name == 'ElasticNet':
                reg = ElasticNet(**self.best_params)
                reg.fit(major_x, major_y) 
                pred_val = reg.predict(minor_x)
                
            elif self.model_name == 'Lasso':
                reg = Lasso(**self.best_params)
                reg.fit(major_x, major_y) 
                pred_val = reg.predict(minor_x)
                
            elif self.model_name == 'ARDR':
                reg = ARDRegression(**self.best_params)
                reg.fit(major_x, major_y) 
                pred_val = reg.predict(minor_x)
                
            elif self.model_name == 'HR':
                reg = HuberRegressor(**self.best_params)
                reg.fit(major_x, major_y) 
                pred_val = reg.predict(minor_x)
                
            elif self.model_name == 'RFR':
                reg = RandomForestRegressor(**self.best_params)
                reg.fit(major_x, major_y) 
                pred_val = reg.predict(minor_x)
                
            elif self.model_name == 'GBM':
                reg = GradientBoostingRegressor(**self.best_params)
                reg.fit(major_x, major_y) 
                pred_val = reg.predict(minor_x)
                
            elif self.model_name == 'Adaboost':
                reg = AdaBoostRegressor(**self.best_params)
                reg.fit(major_x, major_y) 
                pred_val = reg.predict(minor_x)
                
            elif self.model_name == 'Ext':
                reg = ExtraTreesRegressor(**self.best_params)
                reg.fit(major_x, major_y) 
                pred_val = reg.predict(minor_x)
                
            elif RUN_UNIVARIATE:
                if self.model_name == 'AR':
                    reg = statsmodels.tsa.ar_model.AutoReg(major_y,lags = self.best_params['lags']).fit()
                    forecast= reg.predict(pd.to_datetime(minor_y.index.min()),pd.to_datetime(minor_y.index.max()))
                    pred_val = forecast[0]
                    
                elif self.model_name == 'AUTOARIMA':
                    reg = auto_arima(major_y,njobs=-1,trace=False, d=self.best_params['d'])
                    forecast= reg.predict(len(minor_y))
                    pred_val = forecast[-1]
                    
                elif self.model_name == 'ETS':
                    reg = ets.ExponentialSmoothing(major_y, trend=self.best_params['trend']).fit()
                    forecast= reg.predict(len(minor_y))
                    pred_val = forecast[-1]
                    
                elif self.model_name == 'Holt':
                    reg = Holt(major_y, damped=self.best_params['damped'], exponential=self.best_params['exponential']).fit()
                    forecast= reg.predict(len(minor_y))
                    pred_val = forecast[-1]
                    
            else:
                break  
            
            accuracy = 100-(abs(actual_val-pred_val)/actual_val)*100
            model_accuracies.append(accuracy)
            print('Printing Fold Accuracy')
            print('*'*1000)
            print(accuracy)
         
        return [self.cv_error, self.best_params , np.mean(model_accuracies)]
    
    ##################################################################
    def fit_predict(self):
        cv_dict = {}
        ####################################################################
        
        cv_error, best_params , model_acc = self.run_optuna()
        ####################################################################
        if self.model_name == "Ridge":
            best_model = Ridge(alpha=best_params['alpha'],
                        normalize=best_params['normalize'],
                        random_state=self.random_state)
            best_model.fit(self.trainX, self.trainY) 
            forecast = best_model.predict(self.futureX)[0]
            
        elif self.model_name == 'XGB':
            best_model = XGBRegressor(**best_params)
            best_model.fit(self.trainX, self.trainY) 
            forecast = best_model.predict(self.futureX)[0]
            
        elif self.model_name == 'LGBM':
            best_model = LGBMRegressor(random_state=self.random_state,num_parallel_tree=1,objective='regression',
                                        n_estimators=500, n_jobs=-1, verbosity=0, **best_params)
            best_model.fit(self.trainX, self.trainY) 
            forecast = best_model.predict(self.futureX)[0]
            
        elif self.model_name == 'SGDR':
            best_model = SGDRegressor(**best_params)
            best_model.fit(self.trainX, self.trainY) 
            forecast = best_model.predict(self.futureX)[0]
            
        elif self.model_name == 'BR':
            best_model = BayesianRidge(**best_params)
            best_model.fit(self.trainX, self.trainY) 
            forecast = best_model.predict(self.futureX)[0]
            
        elif self.model_name == 'ElasticNet':
            best_model = ElasticNet(**best_params)
            best_model.fit(self.trainX, self.trainY) 
            forecast = best_model.predict(self.futureX)[0]
            
        elif self.model_name == 'Lasso':
            best_model = Lasso(**best_params)
            best_model.fit(self.trainX, self.trainY) 
            forecast = best_model.predict(self.futureX)[0]
            
        elif self.model_name == 'ARDR':
            best_model = ARDRegression(**best_params)
            best_model.fit(self.trainX, self.trainY) 
            forecast = best_model.predict(self.futureX)[0]
            
        elif self.model_name == 'HR':
            best_model = HuberRegressor(**best_params)
            best_model.fit(self.trainX, self.trainY) 
            forecast = best_model.predict(self.futureX)[0]
            
        elif self.model_name == 'RFR':
            best_model = RandomForestRegressor(**best_params)
            best_model.fit(self.trainX, self.trainY) 
            forecast = best_model.predict(self.futureX)[0]
            
        elif self.model_name == 'GBM':
            best_model = GradientBoostingRegressor(**best_params)
            best_model.fit(self.trainX, self.trainY) 
            forecast = best_model.predict(self.futureX)[0]
            
        elif self.model_name == 'Adaboost':
            best_model = AdaBoostRegressor(**best_params)
            best_model.fit(self.trainX, self.trainY) 
            forecast = best_model.predict(self.futureX)[0]
            
        elif self.model_name == 'Ext':
            best_model = ExtraTreesRegressor(**best_params)
            best_model.fit(self.trainX, self.trainY) 
            forecast = best_model.predict(self.futureX)[0]
            
        elif RUN_UNIVARIATE :
            if self.model_name == "AR":
                reg = statsmodels.tsa.ar_model.AutoReg(self.trainY,lags = best_params['lags']).fit()
                forecast= reg.predict(pd.to_datetime(self.futureX.index.min()),pd.to_datetime(self.futureX.index.max())) 
                forecast=forecast[0]
                
            elif self.model_name == "AUTOARIMA":
                reg =auto_arima(self.trainY,njobs=-1,trace=False, d = best_params['d'])
                forecast= reg.predict(len(self.futureX))[-1]
                
            elif self.model_name == 'ETS':
                reg = ets.ExponentialSmoothing(self.trainY, trend=best_params['trend']).fit()
                forecast= reg.predict(len(self.futureX))[-1]
                
            elif self.model_name == 'Holt':
                reg = Holt(self.trainY, damped=best_params['damped'], exponential=best_params['exponential']).fit()
                forecast= reg.predict(len(self.futureX))[-1]
    
        ####################################################################
        if self.y_log_scale:
            forecast = np.exp(forecast) # base
#         else:
#             fut_y = best_model.predict(self.futureX)[0]
#         future_forecast = np.clip(fut_y, self.min_y, self.max_y)
        future_forecast = forecast
        ####################################################################
        error = f'{self.error_metric}'
        cv_dict['model_name'] = self.model_name
        cv_dict['error'] = cv_error
        cv_dict['error_metric'] = self.error_metric
        cv_dict['best_params'] = best_params
        cv_dict['future_forecast'] = future_forecast
        cv_dict['model_acc'] = model_acc
#         cv_dict['test'] = test
#         cv_dict['scaler_info'] = scaler_info
#         cv_dict['normalize_error_info'] = normalize_error_info
#         cv_dict['feature_selection_info']=feature_selection_info
#         cv_dict['top_features'] = self.trainX.columns
        
        return cv_dict      
    ##################################################################
    

In [5]:
def hp_tuning(model,data_obj,lag,target):
    hyper_obj=Hyperparameter(model_name=model,
                        error_metric='mape',
                        trainX=data_obj.trainX.copy(),
                        trainY=data_obj.trainY.copy(),
                        futureX=data_obj.futureX.copy(),
                        test_period=1,backtest_period=BACKTEST_PERIOD,target=target)
    
    train_till = list(obj.trainX.index)[-1]
    forecast_month = list(obj.futureX.index)[0]
    actual = all_feature_df_full[all_feature_df_full.index == forecast_month][country][0]
    a=hyper_obj.fit_predict()
    res=[country,train_till,lag,forecast_month,a['future_forecast'],actual,a['error_metric'],a['error'],
         a['model_name'],a['model_acc'],BACKTEST_PERIOD]
    
    return res



output=[]
if RUN_UNIVARIATE:
    MODEL_LIST = MULTIVARIATE_MODELS + UNIVARIATE_MODELS
else:
    MODEL_LIST = MULTIVARIATE_MODELS
for i,lag in enumerate(FORECAST_PERIOD):
    for j,country in enumerate(ALL_RESPONSE_VAR):
        print('**'*40)
        print(country)
        RESPONSE_VAR = country
        obj = ts_multivariate_func.RunMultivariatePipeline(all_feature_df=all_feature_df,
                                                           response_var=RESPONSE_VAR,
                                                           lag_start=lag,
                                                           add_lag=ADD_LAG,
                                                           add_delta=ADD_DELTA,
                                                           add_rolling_mean=ADD_ROLLING_MEAN,
                                                           cv_threshold=CV_THRESHOLD,
                                                           scaler_info=SCALER_INFO ,
                                                           feature_selection_info =FEATURE_SELECTION_INFO,
                                                           hyperparameter=HYPERPARAMETER,
                                                           normalize_error_info=NORMALIZE_ERROR_INFO,
                                                           remove_self_price=REMOVE_COMMODITY_PRICE,
                                                           dev_range=DEV_RANGE,
                                                           frequency=FREQ,
                                                           commodity=COMMODITY,
                                                           y_log_scale=Y_LOG_SCALE)
        

        result = Parallel(n_jobs=-1)(delayed(hp_tuning)(i, data_obj=obj, lag=lag, target=RESPONSE_VAR) for (i) in list(MODEL_LIST))
        if (i==0) & (j==0):
            output=result
        else:
            output.extend(result)
    
        
columns = ['Country','Training till','Month ahead forecast','Forecast month','Forecast',
           'Actual','Error_metric','Validation error','Model','Model_accuracy','Backtest period']
        
result_df = pd.DataFrame(columns=columns,data=output)

result_df['prediction_accuracy'] = 100-(abs(result_df['Forecast']-result_df['Actual'])/result_df['Actual'])*100

********************************************************************************
China
(29, 50)
For lag 1 >>>>>>>>>>>>>>>>Future Dates are Added
NULLSS Brent Crude Index_EUCRBRDT Index_PX_LAST                 0
Bunker Price Fujairah 380 cst_BUNKFU38 Index_PX_LAST     0
Bunker Price Rotterdam 380 cst_BUNKRD38 Index_PX_LAST    0
Bunker Price Singapore 380 cst_BUNKSI38 Index_PX_LAST    0
Baltic Dry Index_BDIY Index_PX_LAST                      0
Panamax spot_BPIY Index_PX_LAST                          0
 Supramax spot_BSI58A Index_PX_LAST                      0
Capesize spot_BCI14 Index_PX_LAST                        0
Handysize spot_BHSI Index_PX_LAST                        0
Bunker Price Fujairah 180 cst_BUNKFU18 Index_PX_LAST     0
Bunker Price Singapore 180 cst_BUNKSI18 Index_PX_LAST    0
Bunker Price Rotterdam 180 cst_BUNKRD18 Index_PX_LAST    0
WTI  cushing OK spot _USCRWTIC Index_PX_LAST             0
Brent future_CO1 Comdty_PX_LAST                          0
WTI future_CL1 Comdty_

Feature Selection Done
(19, 30)
(1, 30)
DONE!


2022-09-06 09:19:13.284843: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-06 09:19:13.284889: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[I 2022-09-06 09:19:14,800] A new study created in memory with name: starter-experiment
/home/ubuntu/ananya_container_pf/container_env/lib/python3.7/site-packages/optuna/study/study.py:397: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  FutureWarning,
/home/ubuntu/ananya_container_pf/container_env/lib/python3.7/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.St

In [6]:
from datetime import date
def save_result(result,country=None,compiled = True):
    filename = f'FINAL_FORECAST-{date.today()}_test4.xlsx'
    if compiled:
        path = OUTPUT_PATH+"All_results/"
        os.makedirs(path,exist_ok=True)
    else:
        path = OUTPUT_PATH + country+'/'
        os.makedirs(path,exist_ok=True)
    result.to_excel(path+filename,index=False)
    
save_result(result_df,compiled=True)

****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

In [9]:
a=np.array([1,2,3])
b=np.array([1.2,2.1,3])

In [11]:
np.abs((b - a) / a)

array([0.2 , 0.05, 0.  ])

In [20]:
a=7.09035695
b=7.060476
modelling_utility_func.mean_absolute_percentage_error(a,b)

0.4214308279641677

In [16]:
abs((a-b)/a)*100

4.024489019369089

In [19]:
np.mean([99.80027172, 98.8429246, 94.73209721])

97.79176451000001